# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,397896,818980,819063,819255,819304,819308,819330,819518,819594,819765,...,15596515,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [9]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).tocsr(),  # На вход item-user matrix
          show_progress=True)

D:\programs\anaconda3\lib\site-packages\implicit\utils.py:28: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 3.42 s


-----

# Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [10]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекомендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    # data здесь - исходная таблица с данными (data_train)
    def __init__(self, data, weighting='tfidf'):

        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        assert weighting in ['tfidf', 'bm25', 'no'], 'Неверно указан метод'

        self.data = data
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(
            self.user_item_matrix)

        if weighting == 'tfidf':
            self.user_item_matrix = tfidf_weight(self.user_item_matrix)
        elif weighting == 'bm25':
            self.user_item_matrix = bm25_weight(self.user_item_matrix)
        elif weighting != 'no':
            print('Error')

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data, values='quantity'):  # выбор столбца для значений

        # your_code
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id',
                                          columns='item_id',
                                          values=values,
                                          aggfunc='count',
                                          fill_value=0
                                          )

        return user_item_matrix.astype(float)

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=2, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.02, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix.tocsr()))

        return model

    def get_recommendations(self, user, N=5):
        """Рекомендуем топ-N товаров"""

        res = [self.id_to_itemid[rec] for rec in
               self.model.recommend(userid=self.userid_to_id[user],
                                    user_items=self.user_item_matrix.tocsr()[self.userid_to_id[user]],
                                    N=N,
                                    filter_already_liked_items=False,
                                    filter_items=[self.itemid_to_id[999999]],
                                    recalculate_user=True)[0]]
        return res

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        # принимает 1 пользователя
        popularity = (self.data.loc[self.data['user_id'] == user]
                      .groupby(['user_id', 'item_id'])['quantity']
                      .count()
                      .reset_index()
                      .sort_values('quantity', ascending=False)
                      )
        popularity = popularity[popularity['item_id'] != 999999]
        popularity = popularity.head(5).item_id.to_list()
        # если куплено менее 5 товаров
        # для самого частого рекомендуем столько, чтобы добрать до 5
        # чуть лучше метрика
        if len(popularity) < N:
            k = N - len(popularity)
            #             print(user)
            #             print(len(popularity))
            res = [self.id_to_itemid[rec]
                   for rec in self.model.similar_items(self.itemid_to_id[popularity[0]], N=2 + k)[0][1:]]
            res += [self.id_to_itemid[self.model.similar_items(self.itemid_to_id[top_item], N=2)[0][1]]
                    for top_item in popularity[1:]]
        else:
            res = [self.id_to_itemid[self.model.similar_items(self.itemid_to_id[top_item], N=2)[0][1]]
                   for top_item in popularity]

        # или просто отфильтровать пользователей с менее 5 уникальных покупок
        #         res = [self.id_to_itemid[self.model.similar_items(self.itemid_to_id[top_item], N=2)[0][1]]
        #                        for top_item in popularity]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        # товары отсортированы по quantity для исходного товара
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        # your_code
        # упорядочиваем рекомендации, тк own_recommender выдаёт неупорядоченные
#         recs = self.own_recommender.recommend(userid=self.userid_to_id[user],
#                                               user_items=self.user_item_matrix.tocsr()[self.userid_to_id[user]],
#                                               N=N,
#                                               filter_already_liked_items=True,
#                                               filter_items=[self.itemid_to_id[999999]],
#                                               recalculate_user=True)
#         mask = recs[1].argsort()[::-1]  # получим индекс для сортировки
#         res = [self.id_to_itemid[rec] for rec in recs[0][mask]]
#         
        # на самом деле, возвращает упорядоченные рекомендации в версии 0.6.2
        res = [self.id_to_itemid[rec] 
                for rec in self.own_recommender.recommend(userid=self.userid_to_id[user],
                                    user_items=self.user_item_matrix.tocsr()[self.userid_to_id[user]],
                                    N=N - 1, # почему-то даёт рекомендаций на 1 больше
                                    filter_already_liked_items=False,
                                    filter_items=[self.itemid_to_id[999999]],
                                    recalculate_user=True)[0]]
        
#         if len(res) < N:
#             print(user)
#             print(len(res))
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

In [11]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import precision_at_k, recall_at_k, ap_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [12]:
data = pd.read_csv('../raw_data/retail_train.csv')

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [13]:
item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

In [14]:
# проверка prefilter_items()
n_items_before = data_train['item_id'].nunique()

data_train_new = prefilter_items(data=data_train, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_new['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [15]:
# users в test которых нет в train
len(set(data_test['user_id']) - set(data_train_new['user_id']))

8

In [16]:
data_test.loc[~data_test['user_id'].isin(data_train_new['user_id'].unique())].user_id.unique()

array([2120,  954, 2404, 1987,  729,   75, 2364,  650], dtype=int64)

In [17]:
# уберём их
data_test = data_test.loc[data_test['user_id'].isin(data_train_new['user_id'].unique())]

In [18]:
# у 1565 и 1984 менее 5 рек в similar_user! - prefilter_items их пока не отсортировывает - уберём вручную
data_test = data_test.loc[~data_test['user_id'].isin([1565, 1984])]

In [19]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


Проверка модулей

In [20]:
# экземпляр класса
mr = MainRecommender(data_train_new)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [21]:
# подготовленная матрица
mr.user_item_matrix

<2484x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 661080 stored elements in COOrdinate format>

In [22]:
# обученные модели
mr.model, mr.own_recommender

(<implicit.cpu.als.AlternatingLeastSquares at 0xd30002f4f0>,
 <implicit.nearest_neighbours.ItemItemRecommender at 0xd33248b9d0>)

In [23]:
recommends = mr.get_recommendations(user=1)
recommends

[862349, 1075074, 885290, 1051211, 1070820]

In [24]:
# get_similar_items_recommendation() предсказание и метрики
mr.get_similar_items_recommendation(1)

[990335, 5582712, 15926844, 5577022, 982960]

In [25]:
result['similar_recs'] = result['user_id'].apply(lambda x: mr.get_similar_items_recommendation(x))
result.head(2)

,user_id,actual,similar_recs
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[990335, 5582712, 15926844, 5577022, 982960]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1076875, 1092026, 960318, 1075979, 940766]"


In [26]:
# точность
result.apply(lambda row: precision_at_k(row['similar_recs'], row['actual']), axis=1).mean()

0.0854330708661417

In [27]:
# get_similar_users_recommendation() предсказание и метрики
mr.get_similar_users_recommendation(1)

[1070820, 1106523, 856942, 951590, 1126899]

In [28]:
result['similar_users'] = result['user_id'].apply(lambda x: mr.get_similar_users_recommendation(x))
result.head(2)

,user_id,actual,similar_recs,similar_users
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[990335, 5582712, 15926844, 5577022, 982960]","[1070820, 1106523, 856942, 951590, 1126899]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1076875, 1092026, 960318, 1075979, 940766]","[1106523, 951590, 1085604, 9526563, 998206]"


In [29]:
# точность
result.apply(lambda row: precision_at_k(row['similar_users'], row['actual']), axis=1).mean()

0.17755905511810763